In [4]:
import pandas as pd
import numpy as np

## Data Cleaning

In [5]:
df = pd.read_csv("/workspaces/python-for-finance-bobby-zlatarov/DASH.csv")
df["Date"] = pd.to_datetime(df["Date"])
df = df.set_index("Date").sort_index().drop_duplicates()

df.duplicated().sum()

/tmp/ipykernel_10370/120548619.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["Date"] = pd.to_datetime(df["Date"])


np.int64(0)

In [6]:
df["Prev_Close"] = df.Close.shift(1) 
df.Open = df.Open.fillna(df.Prev_Close)

df.Close = df.Close.ffill()

df.High = df.High.fillna(df.groupby(df.index.to_period("M"))["High"].transform("mean"))
df.Low = df.Low.fillna(df.groupby(df.index.to_period("M"))["Low"].transform("mean"))

df.loc[(df.Volume.isna()) & (df.Close == df.Open), "Volume"] = 0

vol_median = df.Volume.median()
mask = df.Volume.isna() & (df.Close != df.Open)
df.loc[mask, "Volume"] = vol_median

df

,Close,High,Low,Open,Volume,Prev_Close
Date,,,,,,
2020-12-09,189.509995,195.500000,163.800003,182.000000,25373700.0,NaN
2020-12-10,186.000000,187.695007,172.636002,179.710007,3506600.0,189.509995
2020-12-11,175.000000,182.000000,168.250000,176.520004,4760600.0,186.000000
2020-12-14,160.000000,170.000000,151.199997,169.100006,7859600.0,175.000000
2020-12-15,158.889999,161.419998,153.759995,157.100006,5017000.0,160.000000
...,...,...,...,...,...,...
2025-06-09,217.490005,219.830002,216.955002,218.029999,2710300.0,218.449997
2025-06-10,214.970001,219.210007,210.927002,216.589996,3916700.0,217.490005
2025-06-11,217.800003,219.529999,212.240005,214.184998,3091500.0,214.970001


## Feature Engineering

**1. Simple Day Returns:** 

Used the `.pct_change()` method to calculate the movement in price from one day to the next.

In [7]:
df["Simple_Return"] = df.Close.pct_change()

**2. Logarithmic Returns:**

To compute them, we used the Previous Close Price and then apply NumPys `log` function. Logarithmic Returns have useful statistic properties to calculate historical returns over multiple periods. 



In [8]:
df["Log_Return"] = np.log(df["Close"] / df["Prev_Close"])

**3. 20-Day Momentum:**

Used the `shift(20)` method to compute the difference in price over 20 days, which helps assess short-term performance and trading momentum.

In [9]:
df["20D_Momentum"] = df["Close"] - df["Close"].shift(20)

**4. 20-Day Simple Moving Average:**

Used the Simple Moving Average to smooth out small variations in the trading data to receive a more accurate picture of the overall trend. The applied 20-Day Moving Average can be considered *fast*.

In [10]:
df["20D_SMA"] = df["Close"].rolling(window=20).mean()

**5. 20-Day Rolling Volatility:**

Computed the Rolling Volatility to determine the degree of variance in the stock. 

In [11]:
df["Dev"] = df["Simple_Return"].rolling(window=20).std()

**6. Day of the Week:**

By computing the specific day related to the Dates, we can observe how financial markets behave differently on different days and explore weekday effects.

In [12]:
df["Day_of_Week"] = df.index.day_name()

**7. Price Surge Identification:**

We define the threshold as when the daily return is more than 4 standard deviations above the mean daily return for the period, which allows us to identify significant price movements.

In [13]:
mean_return = df["Simple_Return"].mean()
std_return = df["Simple_Return"].std()
df["Price_Surge"] = df["Simple_Return"] > (mean_return + 4 * std_return)

**8. Volume Spike Identification:**

We define the threshold as when the trading volume is more than 6 standard deviations above the mean volume for the period, which allows us to identify unusual trading activity. 

In [14]:
mean_volume = df["Volume"].mean()
std_volume = df["Volume"].std()
df["Volume_Spike"] = df["Volume"] > (mean_volume + 6 * std_volume)

**9. Bollinger Bands Calculation:**

Computed the Bollinger Bands, one of the most plotted technical indicators. When the price is near the upper band the security may be overbought, and when the price is near the lower band, the security may be oversold.

In [15]:
df["Upper_BB"] = df["20D_SMA"] + 2 * df["Dev"]
df["Lower_BB"] = df["20D_SMA"] - 2 * df["Dev"]

df

,Close,High,Low,Open,Volume,Prev_Close,Simple_Return,Log_Return,20D_Momentum,20D_SMA,Dev,Day_of_Week,Price_Surge,Volume_Spike,Upper_BB,Lower_BB
Date,,,,,,,,,,,,,,,,
2020-12-09,189.509995,195.500000,163.800003,182.000000,25373700.0,NaN,NaN,NaN,NaN,NaN,NaN,Wednesday,False,False,NaN,NaN
2020-12-10,186.000000,187.695007,172.636002,179.710007,3506600.0,189.509995,-0.018521,-0.018695,NaN,NaN,NaN,Thursday,False,False,NaN,NaN
2020-12-11,175.000000,182.000000,168.250000,176.520004,4760600.0,186.000000,-0.059140,-0.060961,NaN,NaN,NaN,Friday,False,False,NaN,NaN
2020-12-14,160.000000,170.000000,151.199997,169.100006,7859600.0,175.000000,-0.085714,-0.089612,NaN,NaN,NaN,Monday,False,False,NaN,NaN
2020-12-15,158.889999,161.419998,153.759995,157.100006,5017000.0,160.000000,-0.006938,-0.006962,NaN,NaN,NaN,Tuesday,False,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-09,217.490005,219.830002,216.955002,218.029999,2710300.0,218.449997,-0.004395,-0.004404,33.970001,206.409998,0.017753,Monday,False,False,206.445504,206.374493
2025-06-10,214.970001,219.210007,210.927002,216.589996,3916700.0,217.490005,-0.011587,-0.011654,22.869995,207.553498,0.015859,Tuesday,False,False,207.585216,207.521780
2025-06-11,217.800003,219.529999,212.240005,214.184998,3091500.0,214.970001,0.013165,0.013079,20.000000,208.553498,0.014953,Wednesday,False,False,208.583403,208.523593


# Key Dates and Metrics

**1. Highest Rolling Volatility dates**

To identify the date and the value of the highest rolling volatility in each year, we are using the `groupby` function to group the data by the different years for which we also need the `to_period` function.

`df.dropna(subset=["Dev"])`:
Removes rows where the "Dev" (volatility) column is NaN, preventing errors and ensuring only valid data is considered for the maximum search.

`df.index.to_period("Y")`:
Converts the datetime index to a yearly period, so groupby groups by year.

`df.groupby(...["Dev"].idxmax()`:
For each year group, finds the index (date) where "Dev" is the maximum.

`df.loc[...,"Dev"]`:
Selects the rows at those dates and returns the "Dev" value for each year's maximum.


In [16]:
df = df.dropna(subset=["Dev"])

df.loc[df.groupby(df.index.to_period("Y"))["Dev"].idxmax(), "Dev"]

Date
2021-05-20    0.067393
2022-06-01    0.080994
2023-11-06    0.047470
2024-02-27    0.033547
2025-04-10    0.048324
Name: Dev, dtype: float64

**2. Largest Price Surge day**

`df.index.to_period("Y")` converts the datetime index to yearly periods.
This means each group contains all the rows (days) that belong to the same year.

For each year, it finds the index (date) where `Simple_Return` is the highest.
`idxmax()` returns the index label (the date) where the max value occurs within each group/year.

`df.loc[ ... , "Simple_Return"]` selects the rows in df where the index matches the dates found above, and returns only the `Simple_Return` for those rows.

In [17]:
df.loc[df.groupby(df.index.to_period("Y"))["Simple_Return"].idxmax(), "Simple_Return"]

Date
2021-05-14    0.221491
2022-03-16    0.134512
2023-11-02    0.156653
2024-08-02    0.083457
2025-04-09    0.137738
Name: Simple_Return, dtype: float64

**3. Largest Price Drop Day**

`df.groupby(df.index.to_period("Y"))`: Groups the data by year

`["Simple_Return"].idxmin()`: For each year, finds the date where `Simple_Return` is the lowest (most negative)

We use `idxmin()` instead of `idxmax()` because we want the largest price drop (most negative return)

In [18]:


df.loc[df.groupby(df.index.to_period("Y"))["Simple_Return"].idxmin(), "Simple_Return"]

Date
2021-02-22   -0.135457
2022-05-11   -0.131621
2023-02-17   -0.075946
2024-05-02   -0.103170
2025-04-03   -0.080634
Name: Simple_Return, dtype: float64

**4. Highest Volume Day**

`df.groupby(df.index.to_period("Y"))`: Groups the data by year

`["Volume"].idxmax()`: For each year, finds the date where Volume is the highest

In [19]:

df.loc[df.groupby(df.index.to_period("Y"))["Volume"].idxmax(), "Volume"]

Date
2021-03-09    20400000.0
2022-02-17    47405700.0
2023-12-15    30556000.0
2024-05-02    22134800.0
2025-03-21    73637400.0
Name: Volume, dtype: float64

**5. Highest High and Lowest Low**

`groupby(...)[“High”].idxmax()` finds, for each year, the index (i.e., date) where the “High” column is the highest.

`df.loc[ ... , ["High"]]` selects those rows and the “High” column, giving a DataFrame with the date as the index and the highest high value for each year.

Similar to above, but with `idxmin()` to find the date of the minimum (lowest) value in the “Low” column.

`combined =` Combine the two DataFrames (`highs` and `lows`) using their indices (the dates).

`how='outer'` ensures that you get all dates from both DataFrames, even if for some reason a year has a high but not a low (or vice versa).


In [28]:
highs = df.loc[df.groupby(df.index.to_period("Y"))["High"].idxmax(), ["High"]]
lows = df.loc[df.groupby(df.index.to_period("Y"))["Low"].idxmin(), ["Low"]]

highs, "", lows
combined = highs.merge(lows, left_index=True, right_index=True, how='outer')

combined["Year"] = combined.index.to_period("Y").year
combined = combined.reset_index().rename(columns={'index': 'Date'})

combined = combined[["Year", "Date", "High", "Low"]]

print(combined)

   Year       Date        High         Low
0  2021 2021-05-13         NaN  110.129997
1  2021 2021-11-15  257.250000         NaN
2  2022 2022-01-12  152.494995         NaN
3  2022 2022-10-24         NaN   41.365002
4  2023 2023-01-06         NaN   45.931999
5  2023 2023-12-11  103.980003         NaN
6  2024 2024-01-04         NaN   93.330002
7  2024 2024-12-17  181.779999         NaN
8  2025 2025-04-07         NaN  155.399994
9  2025 2025-06-06  220.880005         NaN


**6. Annual Open and Close**

`Open_Date=('Open', lambda x: x.index[0])`:
For the "Open" column, get the index (date) of the first value in each year (i.e., when the year’s opening price occurred).

`Open=('Open', 'first')`:
The opening price at the start of each year (same as before).

`Close_Date=('Close', lambda x: x.index[-1])`:
For the "Close" column, get the index (date) of the last value in each year (i.e., when the year’s closing price occurred).

`Close=('Close', 'last')`:
The closing price at the end of each year (same as before).

`lambda x` functions on the index will capture the specific dates associated with the prices

In [21]:
df.groupby(df.index.to_period('Y')).agg({"Open" : "first", "Close" : "last"})

df.groupby(df.index.to_period('Y')).agg(
    Open_Date=('Open', lambda x: x.index[0]),   
    Open=('Open', 'first'),                      
    Close_Date=('Close', lambda x: x.index[-1]),  
    Close=('Close', 'last')                      
)

,Open_Date,Open,Close_Date,Close
Date,,,,
2021,2021-01-08,152.000000,2021-12-31,148.899994
2022,2022-01-03,149.610001,2022-12-30,48.820000
2023,2023-01-03,49.980000,2023-12-29,98.889999
2024,2024-01-02,97.800003,2024-12-31,167.750000
2025,2025-01-02,168.789993,2025-06-13,218.119995
